# Stochastic Newsvendor Problem

1) Solve the stochastic newsvendor problem

The stochastic newsvendor problem can be modeled as follow: 
       $$Min ~E[ cu-p~min(u,d)] \\
       s.t. ~u>=0
       $$
Here c=1, p=2 and the demand d is spread uniformly over [5,15]
By noting J(u)= cu-p min(u,d) we can determine a more precise expression of J(u).
$$ \begin{align*} 
test \\
J(u) &= E[cu-p~min(u,d)] \\
     &= cu - E[p~min(u,d)]\\
     &= cu - 2u \frac{15-u}{10} - \frac{u^{2}-25}{10}\\
     &= \frac{u^{2}-20u+25}{10}\\
\end{align*}$$

Since we want to minimize J, we find argmin(J)=10 by derivating J. <br>
Therefore the optimal soluition to the stochastic problem is u=10. 

In [2]:
Pkg.add("JuMP")
Pkg.add("Clp") 
using JuMP, Clp

INFO: Package JuMP is already installed
INFO: METADATA is out-of-date — you may not have the latest version of JuMP
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Package Clp is already installed
INFO: METADATA is out-of-date — you may not have the latest version of Clp
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [3]:
function Empcost(N,u)
    c = 1
    p = 2
    totalcost=0
    i=1
    for i = 1:N
        d=rand(50000:150000)
        d=d/10000
        totalcost = totalcost +(c*u - p*min(u,d))
    end
    return totalcost
end

println(Empcost(10000,10)/10000)

-7.469356600000013


3) We are looking for a way to approximate J(10) using the previously defined funciton. Lets determine which N will give us a 95% precision. ( I have no idea what it means). <br>
Usually we use the TCL to compute confidante intervals. <br>
We need to compute the standard deviation of the random variable $cu-p~min(u-d)$ where u=10 and d is uniform over $[5,15]$<br>
$$
\begin{align*}
Var &= E[(-2min(10,d))^2]-E[-2min(10,d)]^2 \\
    &= 4[E[(-2min(10,d))^2]-E[-2min(10,d)]^2] \\
    &= 4[\int_{5}^{10}i^2\frac{1}{10}di + \int_{10}^{15}100\frac{1}{10}di - (-8.75)^2] \\
    &= 4[\frac{875}{30} +50 - \frac{1225}{16}] \\
    &= 4[\frac{1613}{48}] \\
    \\
\sigma &= \frac{\sqrt{125}}{\sqrt{12}} \\
\sigma &approx~3.2275 \\
\end{align*}
$$
Therefore, we can compute a confidance interval: 
$$
\begin{align*}
Confidence ~approx~ [-7.5-1.96\frac{\sigma}{\sqrt{N}},-7.5+1.96\frac{\sigma}{\sqrt{N}}]
\end{align*}
$$
For N=10000 we get around $[-7.5633,-7.4357]$ <br>

I got -7.49414 when running my code, this result sits well into the 95% confidence interval.
$$

In [5]:
    function SAA(N)
    scenario = 5+10*rand(N)

    SOLVER = ClpSolver()  
    m = Model(solver = SOLVER)

    @variable(m,u)
    @variable(m,v[1:N])

    @objective(m, Min, 1/N * sum(u -2*v[i] for i=1:N))
    for i=1:N
     @constraint(m, u >= v[i])
     @constraint(m, scenario[i] >= v[i])
            end
    
     solve(m)
    println(getobjectivevalue(m))
    println("u = ",getvalue(u))
    return getobjectivevalue(m)
end
         SAA(10000)
    

-7.438519427377215
u = 9.890621336306406


-7.438519427377215

5) Show that the SAA value (resp control) almost surely converges toward the optimal value ( resp. control)<br><br>

Let be u, a random real. 
We note $S_{n}(u)=1/N * sum(u -2*min(u,scenario[i]) for i=1:N)$ where scenario[i] is the realization number i of the random variable modeling the demand.<br><br>
Then, the law of large numbers gives that the the suite ~$S_{n}(u)$~ converges toward (u, J(u)=E[u-2min(u,d)]) almost surely. <br><br>
Now, we note $u_{n}$ the optimal control parameter of the N-discretized problem. Therefore, $(u_{n},S_{n}(u_{n}))$~ converges almost certainly toward $(u_{lim},S_{lim})$.  <br><br>
Lets asume $(u_{n},S_{n}(u_{n}))$ does not converges almost certainly toward $(u_{opt},J(u_{opt})$ ~ which is the optimal control and value of the initial problem. Then, there is a non nul probability that either $u_{opt}=/=u_{lim}$ or $J(u_{opt}=/=S_{lim}$ or both. This means by checking either $u_{opt}$ or $u_{lim}$ (lets call it $u_{diff}$ there is a non nul probability that $S_{n}(u_{diff})$ does not converge toward $J(u_{diff})$. <br><br>
This doesn't make sense as we stated previously that for any u, convergence is almost certain. Therefore, we can affirm that $(u_{n},S_{n}(u_{n}))$ converges almost surely toward the optimal control and value. 


In [16]:
saa2=0
saa3=0
saa4=0
for i =1:1000
    saa4= saa4+SAA(4)
    saa2= saa2+SAA(2)
    saa3= saa3+SAA(3)
end
println("saa2=",saa2/1000)
println("saa3=",saa3/1000)
println("saa4=",saa4/1000)

-7.398745112508525
u = 11.615192433572082
-7.981162409464058
u = 12.465613014582202
-10.981248447974021
u = 11.776985811627604
-10.423546681881568
u = 11.862332798393142
-5.236547722993958
u = 8.930562497434721
-8.253039510745287
u = 13.393323050951
-9.774997414702895
u = 10.413558916958191
-7.401732621085159
u = 14.517723977583623
-9.463864558019054
u = 10.012320926111869
-12.993784953893076
u = 13.094099384803588
-6.789179959264516
u = 6.974657870843113
-10.539692231162997
u = 12.387530970584017
-6.336460878269282
u = 14.167630400731273
-5.052066171684606
u = 9.267592552046661
-7.842404005696885
u = 11.211400809412297
-8.306792056696322
u = 12.375402331153872
-11.266899287519358
u = 12.679591939538671
-8.29904664295957
u = 10.225147605645361
-8.861954937013724
u = 9.249847544189887
-6.466976783088989
u = 8.552924856699274
-10.226851850689744
u = 11.37230647797773
-11.826374793537052
u = 14.002999965447199
-7.918327227617259
u = 12.999741087446047
-9.019253117345851
u = 13.04354040396

6) Results: <br>
saa100=-0.7531133073698385 <br>
saa1000=-0.7499344869301525 <br>
saa10000=-0.7495793065699524 <br>

I couldn't manage to get a strict augmentation between the three cases although it seems that "overall" this is indeed the caase. <br>
I also tried with much smaller instances while increasing the number of loops in the Monte-Carlo but couldn't manage to get meaningful results.<br>
I don't know if I am doing something wrong with this one. <br>
<br> 
As for the demonstration, a simple approach would be to argue that we will explore more the demand distrinution. If the probabilty to draw d=10 is 0.5, the rest is continue and so, increasing the number of scenarios increase the informatino at hand to make a decision is more important. The guess is better educated. 